In [1]:
import numpy
import numpy as np
import pandas as pd
import sys, glob
import urllib3
import h5py
import cdsapi, zipfile, os, time
import warnings
import shutil
import xarray
from datetime import date
warnings.filterwarnings('ignore')
# import pycountry
sys.path.append(os.getcwd()+'/../cds-backend/code/')
import cds_eua3 as eua
import numba
import copy
import glob
from numba import njit
import pandas
import glob
import multiprocessing
from functools import partial

In [2]:
c = cdsapi.Client()
r = c.retrieve('insitu-comprehensive-upper-air-observation-network',
               {
                   'variable': ["temperature"],
                   'date': ['20001229','20151230'],
                   'statid': '10393',
#                    'pressure_level':  ['10','20','30','50'],
                   'optional': ['desroziers_30','desroziers_60', 'desroziers_90', 'desroziers_180']
               })
r.download(target='download.zip')
assert os.stat('download.zip').st_size == r.content_length, "Downloaded file is incomplete"
z = zipfile.ZipFile('download.zip')
z.extractall(path='./')
z.close()
cdsfile = glob.glob('./*10393*.nc')

2021-07-21 09:02:40,478 INFO Welcome to the CDS
2021-07-21 09:02:40,479 INFO Sending request to https://sis-dev.climate.copernicus.eu/api/v2/resources/insitu-comprehensive-upper-air-observation-network
2021-07-21 09:02:40,691 INFO Request is completed
2021-07-21 09:02:40,693 INFO Downloading http://136.156.132.176/cache-compute-0000/cache/data2/adaptor.comprehensive_upper_air.retrieve-1626796615.6664505-17472-16-1898f899-94fc-4a6e-ab0a-469fb4bef8cc.zip to download.zip (221.4M)
2021-07-21 09:03:11,848 INFO Download rate 7.1M/s  


In [3]:
c = cdsapi.Client()
r = c.retrieve('insitu-comprehensive-upper-air-observation-network',
               {
                   'variable': ["temperature"],
                   'date': ['20001229','20151230'],
                   'statid': '11035',
#                    'pressure_level':  ['10','20','30','50'],
                   'optional': ['desroziers_30','desroziers_60', 'desroziers_90', 'desroziers_180']
               })
r.download(target='download.zip')
assert os.stat('download.zip').st_size == r.content_length, "Downloaded file is incomplete"
z = zipfile.ZipFile('download.zip')
z.extractall(path='./')
z.close()
cdsfile = glob.glob('./*11035*.nc')

2021-07-21 09:03:13,367 INFO Welcome to the CDS
2021-07-21 09:03:13,368 INFO Sending request to https://sis-dev.climate.copernicus.eu/api/v2/resources/insitu-comprehensive-upper-air-observation-network
2021-07-21 09:03:13,582 INFO Request is completed
2021-07-21 09:03:13,620 INFO Downloading http://136.156.132.176/cache-compute-0000/cache/data2/adaptor.comprehensive_upper_air.retrieve-1626796641.628392-21059-15-0462ec42-ef26-44b6-976f-44777b343438.zip to download.zip (40.8M)
2021-07-21 09:03:21,780 INFO Download rate 5M/s     


In [4]:
lind = glob.glob('./*10393*.nc')[0]
vie = glob.glob('./*11035*.nc')[0]

---

In [5]:
with h5py.File('./merge.nc') as merge:
    name = ['vie', 'lind']
    file = [vie, lind]
    for j in range(len(name)):
            merge.create_group(name[j])
            with h5py.File(file[j], 'r') as filetocopy:
                for i in filetocopy.keys():
                    print(i)
                    merge[name[j]].create_dataset_like(i, filetocopy[i], chunks=True)
                    merge[name[j]][i][:] = filetocopy[i][:]

desroziers_180
desroziers_30
desroziers_60
desroziers_90
lat
lon
obs
plev
report_id
station_id
string11
string25
ta
time
trajectory
trajectory_index
trajectory_label
desroziers_180
desroziers_30
desroziers_60
desroziers_90
lat
lon
obs
plev
report_id
station_id
string11
string20
ta
time
trajectory
trajectory_index
trajectory_label


---

In [13]:
with h5py.File('./merge.nc', 'r') as data:
    print(data)
    print(data.keys())
    for i in data.keys():
        print(i)
        print('---')
        for j in data[i].keys():
            print(j, len(data[i][j][:]))
        
        print('\n')

<HDF5 file "merge.nc" (mode r)>
<KeysViewHDF5 ['lind', 'vie']>
lind
---
desroziers_180 7120490
desroziers_30 7120490
desroziers_60 7120490
desroziers_90 7120490
lat 7120490
lon 7120490
obs 7120490
plev 7120490
report_id 7120490
station_id 7120490
string11 11
string20 20
ta 7120490
time 7120490
trajectory 21901
trajectory_index 7120490
trajectory_label 21901


vie
---
desroziers_180 1072488
desroziers_30 1072488
desroziers_60 1072488
desroziers_90 1072488
lat 1072488
lon 1072488
obs 1072488
plev 1072488
report_id 1072488
station_id 1072488
string11 11
string25 25
ta 1072488
time 1072488
trajectory 11578
trajectory_index 1072488
trajectory_label 11578




In [6]:
def testfunc(file, group):
    with h5py.File(file, 'r') as data:
        print(data[group]['ta'][:])
    return 0

stations = ['vie', 'lind']
file = './merge.nc'
pool = multiprocessing.Pool(processes=2)
func=partial(testfunc, file)
result_list = list(pool.map(func, stations))

[222.7  221.5  217.1  ... 270.39 270.5  270.65]
[220.5  212.5  203.9  ... 269.85 269.84 269.77]
